In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from torchvision.models import EfficientNet_V2_S_Weights
from PIL import Image
import pandas as pd
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm

In [1]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from torchvision.models import EfficientNet_V2_S_Weights
from PIL import Image
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

class GeneralizedStabilityDataset(Dataset):
    def __init__(self, csv_file, img_dir, target_feature, transform=None, augment=False, image_size=224):
        self.stability_data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.augment = augment
        self.image_size = image_size
        self.target_feature = target_feature
        self.augmented_indices = self._create_augmented_indices() if augment else None
        
        # Define the number of classes for each feature
        self.feature_classes = {
            'shapeset': 2,
            'type': 2,
            'total_height': 5,
            'instability_type': 3,
            'cam_angle': 2,
            'stable_height': 6  # Assuming this is the original 'stability_class'
        }
        
        if target_feature not in self.feature_classes:
            raise ValueError(f"Invalid target feature. Choose from {list(self.feature_classes.keys())}")

    def _create_augmented_indices(self):
        base_indices = list(range(len(self.stability_data)))
        flipped_indices = [idx + len(self.stability_data) for idx in base_indices]
        zoomed_indices = [idx + 2 * len(self.stability_data) for idx in base_indices]
        zoomed_flipped_indices = [idx + 3 * len(self.stability_data) for idx in base_indices]
        return base_indices + flipped_indices + zoomed_indices + zoomed_flipped_indices

    def __len__(self):
        return len(self.stability_data) * 4 if self.augment else len(self.stability_data)

    def __getitem__(self, idx):
        if self.augment:
            original_idx = idx % len(self.stability_data)
            augmentation = idx // len(self.stability_data)
        else:
            original_idx = idx
            augmentation = 0

        img_name = str(self.stability_data.iloc[original_idx, 0])
        img_path = os.path.join(self.img_dir, img_name)
        if not os.path.exists(img_path):
            img_path = os.path.join(self.img_dir, f"{img_name}.jpg")
        
        image = Image.open(img_path).convert('RGB')
        
        target_value = self.stability_data.iloc[original_idx][self.target_feature]
        if self.target_feature == 'stability_height':
            target_class = int(target_value) - 1
        else:
            target_class = int(target_value) - 1  # Assuming all features are 1-indexed

        if self.augment:
            if augmentation in [1, 3]:  # Flip
                image = image.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
            if augmentation in [2, 3]:  # Zoom
                width, height = image.size
                crop_size = int(min(width, height) * 0.8)  # Zoom in by 20%
                left = (width - crop_size) // 2
                top = (height - crop_size) // 2
                right = left + crop_size
                bottom = top + crop_size
                image = image.crop((left, top, right, bottom))

        # Resize the image to ensure consistent size
        image = image.resize((self.image_size, self.image_size), Image.BILINEAR)
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(target_class, dtype=torch.long)

class GeneralizedStabilityPredictor(nn.Module):
    def __init__(self, num_classes):
        super(GeneralizedStabilityPredictor, self).__init__()
        weights = EfficientNet_V2_S_Weights.DEFAULT
        self.efficientnet = models.efficientnet_v2_s(weights=weights)
        num_ftrs = self.efficientnet.classifier[1].in_features
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(p=0.3, inplace=True),
            nn.Linear(num_ftrs, num_classes)
        )

    def forward(self, x):
        return self.efficientnet(x)

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, patience=5):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model = None
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        
        # Training phase
        model.train()
        train_loss, train_acc = run_epoch(model, train_loader, criterion, optimizer, device, is_training=True)
        
        # Validation phase
        model.eval()
        val_loss, val_acc = run_epoch(model, val_loader, criterion, optimizer, device, is_training=False)
        
        # Learning rate scheduler step
        scheduler.step(val_loss)

        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        print(f'Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}')
        print('-' * 60)

        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model = model.state_dict()
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience:
            print(f'Early stopping triggered after {epoch + 1} epochs')
            model.load_state_dict(best_model)
            break

    return model

def run_epoch(model, data_loader, criterion, optimizer, device, is_training=True):
    running_loss = 0.0
    correct = 0
    total = 0

    # Create progress bar
    progress_bar = tqdm(data_loader, desc="Training" if is_training else "Validating")

    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        if is_training:
            optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        if is_training:
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # Update progress bar
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100. * correct / total:.2f}%'
        })
    
    epoch_loss = running_loss / len(data_loader.dataset)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc

def calculate_stats(dataset):
    loader = DataLoader(dataset, batch_size=100, num_workers=0, shuffle=False)
    mean = 0.
    std = 0.
    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
    
    mean /= len(dataset)
    std /= len(dataset)
    return mean, std

if __name__ == "__main__":
    features = ['cam_angle', 'instability_type', 'shapeset', 'type', 'total_height']

    for target_feature in features:
        print(f"\n{'='*50}")
        print(f"Training model for feature: {target_feature}")
        print(f"{'='*50}\n")

        # Create full dataset without normalization and augmentation
        full_dataset = GeneralizedStabilityDataset(csv_file='./COMP90086_2024_Project_train/train.csv', 
                                                    img_dir='./COMP90086_2024_Project_train/train', 
                                                    target_feature=target_feature,
                                                    transform=transforms.ToTensor(),
                                                    augment=False,
                                                    image_size=224)

        # Split dataset into train and validation
        val_ratio = 0.1
        dataset_size = len(full_dataset)
        indices = list(range(dataset_size))
        np.random.shuffle(indices)
        split = int(np.floor(val_ratio * dataset_size))
        train_indices, val_indices = indices[split:], indices[:split]

        # Calculate statistics for training and validation sets separately
        train_subset = Subset(full_dataset, train_indices)
        val_subset = Subset(full_dataset, val_indices)

        print("Calculating training dataset statistics...")
        train_mean, train_std = calculate_stats(train_subset)
        print(f"Training dataset mean: {train_mean}")
        print(f"Training dataset std: {train_std}")

        print("Calculating validation dataset statistics...")
        val_mean, val_std = calculate_stats(val_subset)
        print(f"Validation dataset mean: {val_mean}")
        print(f"Validation dataset std: {val_std}")

        # Create separate transforms for training and validation
        train_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=train_mean, std=train_std),
        ])

        val_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=val_mean, std=val_std),
        ])

        # Create augmented training dataset and non-augmented validation dataset
        train_dataset = GeneralizedStabilityDataset(csv_file='./COMP90086_2024_Project_train/train.csv', 
                                                    img_dir='./COMP90086_2024_Project_train/train', 
                                                    target_feature=target_feature,
                                                    transform=train_transform,
                                                    augment=True,
                                                    image_size=224)
        train_dataset = Subset(train_dataset, [i for i in range(len(train_dataset)) if i % len(full_dataset) in train_indices])

        val_dataset = GeneralizedStabilityDataset(csv_file='./COMP90086_2024_Project_train/train.csv', 
                                                    img_dir='./COMP90086_2024_Project_train/train', 
                                                    target_feature=target_feature,
                                                    transform=val_transform,
                                                    augment=False,
                                                    image_size=224)
        val_dataset = Subset(val_dataset, val_indices)

        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

        # Initialize model, criterion, optimizer, and scheduler
        num_classes = full_dataset.feature_classes[target_feature]
        model = GeneralizedStabilityPredictor(num_classes=num_classes)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

        print(f'Training model to predict {target_feature}...')
        model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=30, patience=4)

        torch.save(model.state_dict(), f'stability_predictor_efficientnetv2_{target_feature}.pth')
        print("Training complete. Model saved.")


Training model for feature: cam_angle

Calculating training dataset statistics...
Training dataset mean: tensor([0.4675, 0.4409, 0.4062])
Training dataset std: tensor([0.2722, 0.2287, 0.1914])
Calculating validation dataset statistics...
Validation dataset mean: tensor([0.4703, 0.4435, 0.4091])
Validation dataset std: tensor([0.2709, 0.2268, 0.1901])


c:\Users\bensa\Documents\Git\COMP90086_Project\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Training model to predict cam_angle...
Epoch 1/30


Validating:   4%|▍         | 1/24 [00:00<00:07,  3.10it/s, loss=0.0000, acc=100.00%]


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 7.01 GiB is allocated by PyTorch, and 269.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Loaded dataset mean: tensor([0.4677, 0.4412, 0.4065])
Loaded dataset std: tensor([0.2721, 0.2285, 0.1913])
Splitting dataset into 0.975:0.025 training/test split
Number of classes for each task: {'shapeset': 2, 'stack_type': 2, 'total_height': 5, 'instability_type': 3, 'cam_angle': 2, 'stable_height': 6}
Training...


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [15]:
def predict(model, test_loader, device):
    model.eval()
    predictions = {name: [] for name in model.classifiers.keys()}
    image_ids = []

    with torch.no_grad():
        for inputs, ids in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            for name, output in outputs.items():
                _, preds = torch.max(output, 1)
                predictions[name].extend(preds.cpu().numpy() + 1)  # Add 1 to convert back to 1-6 range
            image_ids.extend(ids.numpy())

    return predictions, image_ids

# Set up device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the unlabeled dataset
test_dataset = StabilityDataset(csv_file='./COMP90086_2024_Project_test/test.csv', 
                                img_dir='./COMP90086_2024_Project_test/test', 
                                transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# Load the trained model
model = StabilityPredictor(num_classes_dict)
model.load_state_dict(torch.load('stability_predictor_efficientnetv2_multi_task.pth'))
model.to(device)

# Make predictions
predictions, image_ids = predict(model, test_loader, device)

# Save predictions to CSV
with open('predictions.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    header = ['id'] + [col if col != 'stack_type' else 'type' for col in predictions.keys()]
    writer.writerow(header)
    for i, img_id in enumerate(image_ids):
        row = [int(img_id) + 1]  # Assuming image IDs start from 0
        for name in predictions.keys():
            pred = int(predictions[name][i])
            if name in ['shapeset', 'type', 'cam_angle', 'stable_height']:
                pred += 1
            elif name == 'total_height':
                pred += 2
            # 'instability_type' doesn't need adjustment
            row.append(pred)
        writer.writerow(row)

print("Predictions saved to predictions.csv")


Loaded dataset mean: tensor([0.4677, 0.4412, 0.4065])
Loaded dataset std: tensor([0.2721, 0.2285, 0.1913])
shapeset: min = 1, max = 2, unique values = [1, 2]
type: min = 1, max = 2, unique values = [1, 2]
total_height: min = 2, max = 6, unique values = [2, 3, 4, 5, 6]
instability_type: min = 0, max = 2, unique values = [0, 1, 2]
cam_angle: min = 1, max = 2, unique values = [1, 2]
stable_height: min = 1, max = 6, unique values = [1, 2, 3, 4, 5, 6]
Splitting dataset into 0.975:0.025 training/test split
Training...


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
